In [1]:
from aiida import load_profile
aiida_profile = load_profile()
aiida_profile.name

'default'

In [2]:
## Loading the some require packages and module
from aiida_kkr.workflows import (combine_imps_wc, kkr_flex_wc,
                                kkr_imp_wc, kkr_imp_sub_wc
                                )

from aiida.orm import (Group, load_group, load_node, Dict,
                      )
from aiida.engine import submit

In [3]:
builder = combine_imps_wc.get_builder()

In [4]:
group_64 = load_group(64)
## first node of group-64 is for kkr_imp_wc process
# is being used for the host_gf or kkr_flex files
imp_wc_with_gf = group_64.nodes[0]
imp1_wc = group_64.nodes[2]
imp2_wc = group_64.nodes[2]

gf_host_remote = imp_wc_with_gf.outputs.remote_data_gf
host_gf_wc = imp_wc_with_gf.get_outgoing(node_class=kkr_flex_wc).all()[0].node


impurity1_output_node = imp1_wc.outputs.workflow_info
impurity2_output_node = imp2_wc.outputs.workflow_info
offset_imp2 = {'index':1}

In [10]:
scf_node = imp1_wc.get_outgoing(node_class=kkr_imp_sub_wc).all()[0].node
scf ={'kkrimp':scf_node.inputs.kkrimp, 
      'options':scf_node.inputs.options,
      'wf_parameters':scf_node.inputs.wf_parameters}

host_gf = {'kkr':host_gf_wc.inputs.kkr,
          'options': host_gf_wc.inputs.options,
          'params_kkr_overwrite':host_gf_wc.inputs.wf_parameters}

In [19]:
host_gf_wc

<WorkChainNode: uuid: 5c3c6763-3796-4447-b095-22b516442089 (pk: 10390) (aiida.workflows:kkr.gf_writeout)>

In [16]:
imp1_wc

<WorkChainNode: uuid: 6ef52bbc-e884-4d75-bc45-0163a5c81661 (pk: 10725) (aiida.workflows:kkr.imp)>

In [8]:
builder.impurity1_output_node = impurity1_output_node
builder.impurity2_output_node = impurity2_output_node
builder.offset_imp2 = Dict(dict=offset_imp2)
# builder.metadata.description = 
# builder.scf.kkrimp = 
# builder.
builder.scf = scf 


builder.host_gf = host_gf
# builder.gf_host_remote = gf_host_remote
builder.metadata.label = str(imp1_wc.label+ ':' +imp2_wc.label)

In [13]:
# combined_imp_submission = submit(builder)